In [ ]:
import os, re
import fitz as pymupdf
from deep_translator import GoogleTranslator
from operator import itemgetter
from pdf2docx import Converter
import docx

In [ ]:
# === Configuration ===
doc_folder = r"c:\Users\f.gionnane\Documents\Document Processing Projects\PDF Translation"
file_name = "Charles-Gave-Un-Libéral-Nommé-Jésus.pdf"
filename_without_extension = os.path.splitext(file_name)[0]
path_to_doc = os.path.join(doc_folder, file_name)

target_es = "es"
es_translator = GoogleTranslator(source='auto', target=target_es)

# === Ouvrir le document PDF ===
doc = pymupdf.open(path_to_doc)

In [ ]:
print("Processing...")

# Track the progress with print statements
for page_num, page in enumerate(doc, start=1):
    print(f"Processing page {page_num}/{len(doc)}...")
    
    blocks = page.get_text("blocks", flags=pymupdf.TEXT_DEHYPHENATE)
    
    for block_num, block in enumerate(blocks, start=1):
        bbox = block[:4]
        original = block[4]
        
        print(f"  Processing block {block_num}/{len(blocks)} on page {page_num}...")

        # Translate and ensure it's a string
        translated = str(es_translator.translate(original))  # Ensure it's a string
        
        # Check if translation is successful
        if translated:
            print(f"    Translation successful: {translated[:30]}...")  # Print part of translation for verification
        else:
            print(f"    Translation failed or is empty for block {block_num}")

        # Clear the original block's area and insert the translated text
        page.draw_rect(bbox, color=None, fill=pymupdf.pdfcolor["white"])
        page.insert_htmlbox(bbox, translated)

# Output file path
output_path = os.path.join(doc_folder, f"{filename_without_extension}_{target_es}.pdf")
print(f"Saving translated document to {output_path}...")

# Save the document
doc.save(output_path)

print("Processing complete!")

test

In [ ]:
print("📄 Début du traitement du document...")

# === Créer le fichier texte pour le suivi ===
txt_output_path = os.path.join(doc_folder, f"{filename_without_extension}_{target_es}_draft.txt")
with open(txt_output_path, "w", encoding="utf-8") as txt_file:
    txt_file.write(f"Traduction du document : {file_name} vers [{target_es}]\n\n")

# === Fonction pour regrouper les blocs en paragraphes ===
def group_blocks_by_y(blocks, y_threshold=5):
    blocks.sort(key=itemgetter(1))  # Trier selon la coordonnée Y (top)
    grouped = []
    current_group = []
    last_bottom = 0

    for block in blocks:
        top = block[1]
        if not current_group or top - last_bottom <= y_threshold:
            current_group.append(block)
            last_bottom = block[3]
        else:
            grouped.append(current_group)
            current_group = [block]
            last_bottom = block[3]

    if current_group:
        grouped.append(current_group)

    return grouped

# === Fonction pour nettoyer le texte corrompu ===
def clean_text(text):
    # Enlever les caractères non valides ou corrompus (comme les caractères '�')
    # Cela va remplacer les caractères indésirables par des espaces ou les ignorer
    cleaned = re.sub(r'[^\x00-\x7F]+', '', text)  # Supprime les caractères non-ASCII
    return cleaned

# === Traitement des pages ===
for page_num, page in enumerate(doc, start=1):
    print(f"\n➡️  Page {page_num}/{len(doc)} en cours...")

    blocks = page.get_text("blocks", flags=pymupdf.TEXT_DEHYPHENATE)
    paragraph_groups = group_blocks_by_y(blocks)

    with open(txt_output_path, "a", encoding="utf-8") as txt_file:
        txt_file.write(f"\n{'='*40}\nPage {page_num}\n{'='*40}\n")

    for group_num, block_group in enumerate(paragraph_groups, start=1):
        paragraph_text = " ".join(block[4].strip() for block in block_group if block[4].strip())
        if not paragraph_text:
            continue

        # Nettoyer le texte corrompu
        cleaned_text = clean_text(paragraph_text)

        # Si après nettoyage, le texte est vide (c'est-à-dire corrompu ou illisible), on l'ignore
        if not cleaned_text.strip():
            print(f"    ⚠️ Paragraphe {group_num} ignoré, texte corrompu ou illisible.")
            continue

        print(f"  ↪️ Paragraphe {group_num} : {cleaned_text[:50]}...")

        try:
            translated = str(es_translator.translate(cleaned_text))
            if not translated.strip():
                print(f"    ⚠️ Traduction vide")
                continue
        except Exception as e:
            print(f"    ❌ Erreur de traduction : {e}")
            continue

        print(f"    ✅ Traduction : {translated[:50]}...")

        # Écrire uniquement la traduction dans le fichier texte
        with open(txt_output_path, "a", encoding="utf-8") as txt_file:
            txt_file.write(f"\n--- Paragraphe {group_num} ---\n")
            txt_file.write(f"[Traduction]\n{translated}\n")

        # Calculer la bbox englobante
        x0 = min(block[0] for block in block_group)
        y0 = min(block[1] for block in block_group)
        x1 = max(block[2] for block in block_group)
        y1 = max(block[3] for block in block_group)
        padding = 1
        bbox = (x0 + padding, y0 + padding, x1 - padding, y1 - padding)

        # Nettoyage + insertion dans le PDF
        page.draw_rect(bbox, color=None, fill=pymupdf.pdfcolor["white"])
        page.insert_htmlbox(bbox, translated)

# === Sauvegarde finale du PDF ===
output_path = os.path.join(doc_folder, f"{filename_without_extension}_{target_es}.pdf")
print(f"\n💾 Sauvegarde du PDF traduit dans :\n{output_path}")
doc.save(output_path)

print(f"\n📝 Texte de traduction temporaire dans :\n{txt_output_path}")
print("✅ Traitement terminé !")


Test tranlate with docx conversion

In [ ]:
import os
from pdf2docx import Converter

# Chemin du PDF et du DOCX
doc_folder = r"c:\Users\f.gionnane\Documents\Document Processing Projects\PDF Translation"
file_name = "Charles-Gave-Un-Libéral-Nommé-Jésus.pdf"  # Remplacez par le nom de votre fichier PDF
filename_without_extension = os.path.splitext(file_name)[0]

# Chemin complet pour le fichier PDF et le fichier DOCX converti
pdf_path = os.path.join(doc_folder, file_name)
docx_path = os.path.join(doc_folder, f"{filename_without_extension}.docx")

# Convertir le PDF en DOCX
cv = Converter(pdf_path)
cv.convert(docx_path, start=0, end=None)

print(f"Le fichier PDF a été converti en DOCX et sauvegardé sous : {docx_path}")


In [16]:
import os
import docx
from deep_translator import GoogleTranslator

# Initialiser le traducteur
target_es = "es"
es_translator = GoogleTranslator(source='auto', target=target_es)
print(f"Traducteur initialisé pour traduire en {target_es}.")

# Chemin du fichier DOCX
docx_path = os.path.join(doc_folder, f"{filename_without_extension}.docx")
print(f"Chargement du document DOCX à partir de : {docx_path}")
doc = docx.Document(docx_path)
print(f"Document DOCX chargé avec {len(doc.paragraphs)} paragraphes.")

# Chemin du fichier TXT pour prévisualiser la traduction
txt_preview_path = os.path.join(doc_folder, f"{filename_without_extension}_{target_es}_preview.txt")

# Créer/vider le fichier texte avant d’écrire dedans
with open(txt_preview_path, "w", encoding="utf-8") as preview_file:
    preview_file.write(f"Prévisualisation de la traduction en espagnol de : {file_name}\n\n")

# Traduction du texte dans le DOCX
for index, para in enumerate(doc.paragraphs, start=1):
    original_text = para.text.strip()

    if original_text:
        print(f"[{index}] Traduction du texte : {original_text[:50]}...")
        try:
            translated_text = es_translator.translate(original_text)
            para.text = translated_text

            # Écriture dans le fichier .txt
            with open(txt_preview_path, "a", encoding="utf-8") as preview_file:
                preview_file.write(translated_text + "\n")

            print(f"[{index}] ✅ Texte traduit ajouté au fichier .txt")

        except Exception as e:
            print(f"[{index}] ❌ Erreur de traduction : {e}")
    else:
        print(f"[{index}] (Paragraphe vide ignoré)")

# Sauvegarde du document DOCX traduit
docx_output_path = os.path.join(doc_folder, f"{filename_without_extension}_{target_es}.docx")
doc.save(docx_output_path)
print(f"\n✅ Le document traduit a été sauvegardé sous : {docx_output_path}")
print(f"📄 Fichier de prévisualisation disponible ici : {txt_preview_path}")


Traducteur initialisé pour traduire en es.
Chargement du document DOCX à partir de : c:\Users\f.gionnane\Documents\Document Processing Projects\PDF Translation\Charles-Gave-Un-Libéral-Nommé-Jésus.docx
Document DOCX chargé avec 1380 paragraphes.
[1] Traduction du texte : Charles Gave 
Un Libéral Nommé Jésus Parabole Écon...
[1] ✅ Texte traduit ajouté au fichier .txt
[2] Traduction du texte : 1...
[2] ✅ Texte traduit ajouté au fichier .txt
[3] (Paragraphe vide ignoré)
[4] (Paragraphe vide ignoré)
[5] (Paragraphe vide ignoré)
[6] (Paragraphe vide ignoré)
[7] Traduction du texte : Cet ouvrage vous est offert par :...
[7] ✅ Texte traduit ajouté au fichier .txt
[8] (Paragraphe vide ignoré)
[9] (Paragraphe vide ignoré)
[10] (Paragraphe vide ignoré)
[11] Traduction du texte : Sommaire...
[11] ✅ Texte traduit ajouté au fichier .txt
[12] Traduction du texte : AVANT-PROPOS 	 p.7 Nous sommes des nains sur les é...
[12] ✅ Texte traduit ajouté au fichier .txt
[13] Traduction du texte : CHAPITRE I 	 